<a href="https://colab.research.google.com/github/yoonju977/Digital-Image-Processing/blob/main/Lab08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
imgpath = '/content/drive/MyDrive/HUFS/24-2의료영상처리학/DIP1/DIP4E'

Mounted at /content/drive


In [2]:
!python -m pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 45.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 49.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117

In [3]:
import radiomics
from radiomics import featureextractor
import matplotlib.pyplot as plt
import SimpleITK as sitk
import nibabel as nib
import pandas as pd

In [4]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1]
settings['normalize']=True
settings['geometryTolerance']=1
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call
# Enable a filter (in addition to the 'Original' filter already enabled)
#extractor.enableImageTypeByName('LoG')
#extractor.enableImageTypeByName('Wavelet')
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('glcm')
extractor.enabledFeatures
print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters


INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'binWidth': 20, 'sigma': [1], 'normalize': True, 'geometryTolerance': 1}


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1], 'geometryTolerance': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'shape': [], 'glcm': []}


In [5]:
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('glcm')
extractor.enabledFeatures

{'firstorder': [], 'shape': [], 'glcm': []}

In [7]:
h1 = nib.load('/content/drive/MyDrive/HUFS/24-2의료영상처리학/DIP1/DIP4E/H_t1ce.nii.gz')
img = h1.get_fdata()
img.shape

(240, 240, 155)

In [8]:
h2 = nib.load('/content/drive/MyDrive/HUFS/24-2의료영상처리학/DIP1/DIP4E/H_seg.nii.gz')
seg = h2.get_fdata()
seg.shape

(240, 240, 155)

In [9]:
image = sitk.GetImageFromArray(img)
mask = sitk.GetImageFromArray(seg)

In [10]:
features = extractor.execute(image,mask)

INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [11]:
feature_names = list((filter(lambda k: k.startswith("original_"),features)))

In [12]:
df1 = pd.DataFrame()
for j in range(len(feature_names)):
  print(j,feature_names[j],features[feature_names[j]])
  new_row = {"feature_name":feature_names[j], "H":features[feature_names[j]]}
  df1 = pd.concat([df1, pd.DataFrame([new_row])], ignore_index=True)

0 original_shape_Elongation 0.7705221899156938
1 original_shape_Flatness 0.4456204321340164
2 original_shape_LeastAxisLength 8.942992981667214
3 original_shape_MajorAxisLength 20.068633161276786
4 original_shape_Maximum2DDiameterColumn 25.553864678361276
5 original_shape_Maximum2DDiameterRow 20.0
6 original_shape_Maximum2DDiameterSlice 21.840329667841555
7 original_shape_Maximum3DDiameter 27.910571473905726
8 original_shape_MeshVolume 339.4583333333333
9 original_shape_MinorAxisLength 15.463327172041701
10 original_shape_Sphericity 0.3280588293045878
11 original_shape_SurfaceArea 717.3372388711193
12 original_shape_SurfaceVolumeRatio 2.113181997410932
13 original_shape_VoxelVolume 413.0
14 original_firstorder_10Percentile 1.7756253364023524
15 original_firstorder_90Percentile 2.8247011799850297
16 original_firstorder_Energy 2380.4530635283995
17 original_firstorder_Entropy -3.203426503814917e-16
18 original_firstorder_InterquartileRange 0.5057083553680597
19 original_firstorder_Kurtosi

In [13]:
df1

,feature_name,H
0,original_shape_Elongation,0.770522
1,original_shape_Flatness,0.44562
2,original_shape_LeastAxisLength,8.942993
3,original_shape_MajorAxisLength,20.068633
4,original_shape_Maximum2DDiameterColumn,25.553864678361276
5,original_shape_Maximum2DDiameterRow,20.0
6,original_shape_Maximum2DDiameterSlice,21.840329667841555
7,original_shape_Maximum3DDiameter,27.910571473905726
8,original_shape_MeshVolume,339.4583333333333
9,original_shape_MinorAxisLength,15.463327


In [14]:
df1.to_csv("H_features.csv")

In [49]:
import pandas as pd

def select_top_features_from_csv(file_path, n=5, output_csv_path=None):
    """
    Select the top N features with the largest differences from a CSV file.
    :param file_path: Path to the CSV file.
    :param n: Number of top features to select based on absolute values.
    :return: DataFrame of top N features.
    """
    # Load CSV into DataFrame
    df = pd.read_csv(file_path)

    # Ensure the CSV contains the necessary columns
    if "feature_name" not in df.columns or "H" not in df.columns:
        raise ValueError("CSV file must contain 'feature_name' and 'H' columns.")

    # Calculate absolute values
    df["AbsValue"] = df["H"].abs()

    # Sort by absolute value differences
    df_sorted = df.sort_values(by="AbsValue", ascending=False)

    # Select top N features
    top_features = df_sorted.head(n)

    # Save to a new CSV if a path is provided
    if output_csv_path:
        top_features.to_csv(output_csv_path, index=False)

    return top_features

# Example: Using a CSV file 'H_features.csv'
file_path = "H_features.csv"
output_csv_path = "H_Top_Features.csv"
# Generate top features as a DataFrame
df3 = select_top_features_from_csv(file_path, n=5, output_csv_path=output_csv_path)

# Print DataFrame
print("Top 5 Features DataFrame (df3):")
print(df3)

Top 5 Features DataFrame (df3):
    Unnamed: 0                     feature_name            H     AbsValue
16          16       original_firstorder_Energy  2380.453064  2380.453064
29          29  original_firstorder_TotalEnergy  2380.453064  2380.453064
11          11       original_shape_SurfaceArea   717.337239   717.337239
13          13       original_shape_VoxelVolume   413.000000   413.000000
8            8        original_shape_MeshVolume   339.458333   339.458333


1. original_firstorder_Energy

- 계산 방법: ROI(region of interest) 내 intensity 값의 확률 분포, 값이 높을수록 voxel intensity의 변동성이 적고, 조화로운 intensity 분포를 나타낸다.
- 의미: Energy는 종양 영역에서 intensity 값의 균일성을 나타내며, 높은 Energy 값은 일정한 밀도 또는 균일한 구조를 의미하며, 조직의 물리적 특성과 밀접하게 연결.

2. original_firstorder_TotalEnergy

- 계산 방법: 모든 voxel intensity의 제곱 합.
- 의미:TotalEnergy는 ROI 내 전체 intensity의 크기를 나타낸다. 높은 값은 높은 총 신호 강도를 의미하며, 조직의 구성 또는 병변의 대사를 나타낼 수 있다.

3. original_shape_SurfaceArea
- 계산 방법: SurfaceArea는 3D 종양 형태에서 표면적을 계산. Mesh 모델 기반의 표면 삼각형으로 계산된다.
- 의미:종양의 표면 복잡성을 나타낸다.  큰 표면적은 불규칙하고 더 복잡한 모양을 나타내며, 침습적인 종양일 가능성이 높다.

4. original_shape_VoxelVolume
- 계산 방법: ROI를 구성하는 voxel의 총 개수와 크기를 곱해 계산.
- 의미: VoxelVolume은 종양의 전체 크기 또는 부피를 나타낸다. 종양의 성장이나 크기 변화를 측정하는 중요한 지표.

5. original_shape_MeshVolume
- 계산 방법: MeshVolume은 3D로 재구성된 종양의 표면 모델에서 계산한 부피. 종양의 경계(mesh)를 기반으로 삼각형 계산을 통해 도출됨.
- 의미: MeshVolume은 종양의 실제 3D 부피를 나타내며 이 값은 병변의 확산이나 축소를 평가하는 데 유용하다.


In [36]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1]
settings['normalize']=True
settings['geometryTolerance']=1
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call
# Enable a filter (in addition to the 'Original' filter already enabled)
#extractor.enableImageTypeByName('LoG')
#extractor.enableImageTypeByName('Wavelet')
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('glcm')
extractor.enabledFeatures
print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'binWidth': 20, 'sigma': [1], 'normalize': True, 'geometryTolerance': 1}


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1], 'geometryTolerance': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'shape': [], 'glcm': []}


In [37]:
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
extractor.enableFeatureClassByName('shape')
extractor.enableFeatureClassByName('glcm')
extractor.enabledFeatures

{'firstorder': [], 'shape': [], 'glcm': []}

In [38]:
L1 = nib.load('/content/drive/MyDrive/HUFS/24-2의료영상처리학/DIP1/DIP4E/L_t1ce.nii.gz')
img = L1.get_fdata()
img.shape

(240, 240, 155)

In [39]:
L2 = nib.load('/content/drive/MyDrive/HUFS/24-2의료영상처리학/DIP1/DIP4E/L_seg.nii.gz')
seg = L2.get_fdata()
seg.shape

(240, 240, 155)

In [40]:
image = sitk.GetImageFromArray(img)
mask = sitk.GetImageFromArray(seg)

In [41]:
features = extractor.execute(image,mask)

INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [42]:
feature_names = list((filter(lambda k: k.startswith("original_"),features)))

In [43]:
df2 = pd.DataFrame()
for j in range(len(feature_names)):
  print(j,feature_names[j],features[feature_names[j]])
  new_row = {"feature_name":feature_names[j], "L":features[feature_names[j]]}
  df2 = pd.concat([df2, pd.DataFrame([new_row])], ignore_index=True)

0 original_shape_Elongation 0.7533119796040892
1 original_shape_Flatness 0.6443580138680813
2 original_shape_LeastAxisLength 37.5758429212442
3 original_shape_MajorAxisLength 58.3151634844679
4 original_shape_Maximum2DDiameterColumn 52.40229002629561
5 original_shape_Maximum2DDiameterRow 62.297672508690084
6 original_shape_Maximum2DDiameterSlice 66.48308055437865
7 original_shape_Maximum3DDiameter 70.68946173228369
8 original_shape_MeshVolume 50105.125
9 original_shape_MinorAxisLength 43.92951124542062
10 original_shape_Sphericity 0.4696461162538371
11 original_shape_SurfaceArea 13994.847030953128
12 original_shape_SurfaceVolumeRatio 0.2793096919916501
13 original_shape_VoxelVolume 50041.0
14 original_firstorder_10Percentile 1.3031418316020564
15 original_firstorder_90Percentile 2.159491281085587
16 original_firstorder_Energy 151226.8685396885
17 original_firstorder_Entropy 0.0014719447920251048
18 original_firstorder_InterquartileRange 0.4281747247417651
19 original_firstorder_Kurtosi

In [44]:
df2

,feature_name,L
0,original_shape_Elongation,0.753312
1,original_shape_Flatness,0.644358
2,original_shape_LeastAxisLength,37.575843
3,original_shape_MajorAxisLength,58.315163
4,original_shape_Maximum2DDiameterColumn,52.40229002629561
5,original_shape_Maximum2DDiameterRow,62.297672508690084
6,original_shape_Maximum2DDiameterSlice,66.48308055437865
7,original_shape_Maximum3DDiameter,70.68946173228369
8,original_shape_MeshVolume,50105.125
9,original_shape_MinorAxisLength,43.929511


In [45]:
df2.to_csv("L_features.csv")

In [50]:
import pandas as pd

def select_top_features_from_csv(file_path, n=5, output_csv_path=None):
    """
    Select the top N features with the largest differences from a CSV file.
    :param file_path: Path to the CSV file.
    :param n: Number of top features to select based on absolute values.
    :return: DataFrame of top N features.
    """
    # Load CSV into DataFrame
    df = pd.read_csv(file_path)

    # Ensure the CSV contains the necessary columns
    if "feature_name" not in df.columns or "L" not in df.columns:
        raise ValueError("CSV file must contain 'feature_name' and 'L' columns.")

    # Calculate absolute values
    df["AbsValue"] = df["L"].abs()

    # Sort by absolute value differences
    df_sorted = df.sort_values(by="AbsValue", ascending=False)

    # Select top N features
    top_features = df_sorted.head(n)

    # Save to a new CSV if a path is provided
    if output_csv_path:
        top_features.to_csv(output_csv_path, index=False)

    return top_features


file_path = "L_features.csv"
output_csv_path = "L_Top_Features.csv"

# Generate top features as a DataFrame
df4 = select_top_features_from_csv(file_path, n=5, output_csv_path=output_csv_path)

# Print DataFrame
print("Top 5 Features DataFrame (df4):")
print(df4)

Top 5 Features DataFrame (df4):
    Unnamed: 0                     feature_name              L       AbsValue
16          16       original_firstorder_Energy  151226.868540  151226.868540
29          29  original_firstorder_TotalEnergy  151226.868540  151226.868540
8            8        original_shape_MeshVolume   50105.125000   50105.125000
13          13       original_shape_VoxelVolume   50041.000000   50041.000000
11          11       original_shape_SurfaceArea   13994.847031   13994.847031


In [53]:
print("Highgrade glioma_Top 5 Features DataFrame (df3):")
print(df3)
print("Lowgrade gliomaTop 5 Features DataFrame (df4):")
print(df4)

Highgrade glioma_Top 5 Features DataFrame (df3):
    Unnamed: 0                     feature_name            H     AbsValue
16          16       original_firstorder_Energy  2380.453064  2380.453064
29          29  original_firstorder_TotalEnergy  2380.453064  2380.453064
11          11       original_shape_SurfaceArea   717.337239   717.337239
13          13       original_shape_VoxelVolume   413.000000   413.000000
8            8        original_shape_MeshVolume   339.458333   339.458333
Lowgrade gliomaTop 5 Features DataFrame (df4):
    Unnamed: 0                     feature_name              L       AbsValue
16          16       original_firstorder_Energy  151226.868540  151226.868540
29          29  original_firstorder_TotalEnergy  151226.868540  151226.868540
8            8        original_shape_MeshVolume   50105.125000   50105.125000
13          13       original_shape_VoxelVolume   50041.000000   50041.000000
11          11       original_shape_SurfaceArea   13994.847031   13994